<a href="https://colab.research.google.com/github/Ehsan1997/ObjectDetection-FRCNN-YOLO-Retina/blob/master/YOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
zip_path = '/content/drive/My Drive/Datasets/SIMS-dataset.zip'
!unzip '$zip_path' -d ./data/
!unzip -q 'data/SIMS dataset/images.zip' -d data/
!unzip -q '/content/drive/My Drive/Datasets/SIMS_Custom_Annotations/pascal_voc_sims.zip'
!rm  data/images/*.txt

Archive:  /content/drive/My Drive/Datasets/SIMS-dataset.zip
  inflating: ./data/SIMS dataset/test.txt  
  inflating: ./data/SIMS dataset/validation.txt  
  inflating: ./data/SIMS dataset/training.txt  
  inflating: ./data/SIMS dataset/Assignment-2_modified scope.pdf  
  inflating: ./data/SIMS dataset/Annotations_in_3_formats.zip  
  inflating: ./data/SIMS dataset/images.zip  


In [3]:
!git clone https://github.com/experiencor/keras-yolo3
!mv keras-yolo3/* .
!rm -r keras-yolo3
# !pip install -r requirements.txt

Cloning into 'keras-yolo3'...
remote: Enumerating objects: 220, done.
remote: Total 220 (delta 0), reused 0 (delta 0), pack-reused 220
Receiving objects: 100% (220/220), 91.26 KiB | 1.94 MiB/s, done.
Resolving deltas: 100% (118/118), done.


In [4]:
import os
import numpy as np
import json
from voc import parse_voc_annotation
from yolo import create_yolov3_model, dummy_loss
from generator import BatchGenerator
from utils.utils import normalize, evaluate, makedirs
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam
from callbacks import CustomModelCheckpoint, CustomTensorBoard
from utils.multi_gpu_model import multi_gpu_model
import tensorflow as tf
import keras
from keras.models import load_model
from keras.callbacks import CSVLogger

Using TensorFlow backend.


In [5]:
!rm -r /content/data/yolo_val

rm: cannot remove '/content/data/yolo_val': No such file or directory


In [0]:
!mkdir /content/data/yolo_val
!mkdir /content/data/yolo_val/images
!mkdir /content/data/yolo_val/annots

In [7]:
!bash ./create_validation_set.sh /content/data/images /content/PASCALVOCAnnots/ 1700 /content/data/yolo_val

moving files 1069.jpg 1069.xml
moving files 1990.jpg 1990.xml
moving files 4722.jpg 4722.xml
moving files 4032.jpg 4032.xml
moving files 0876.jpg 0876.xml
moving files 1305.jpg 1305.xml
moving files 2886.jpg 2886.xml
moving files 3114.jpg 3114.xml
moving files 0225.jpg 0225.xml
moving files 3516.jpg 3516.xml
moving files 3507.jpg 3507.xml
moving files 3230.jpg 3230.xml
moving files 0135.jpg 0135.xml
moving files 1316.jpg 1316.xml
moving files 4037.jpg 4037.xml
moving files 2792.jpg 2792.xml
moving files 1935.jpg 1935.xml
moving files 1798.jpg 1798.xml
moving files 1220.jpg 1220.xml
moving files 1942.jpg 1942.xml
moving files 3759.jpg 3759.xml
moving files 0216.jpg 0216.xml
moving files 1626.jpg 1626.xml
moving files 3779.jpg 3779.xml
moving files 4803.jpg 4803.xml
moving files 0189.jpg 0189.xml
moving files 2899.jpg 2899.xml
moving files 0705.jpg 0705.xml
moving files 3292.jpg 3292.xml
moving files 1917.jpg 1917.xml
moving files 3197.jpg 3197.xml
moving files 3146.jpg 3146.xml
moving f

In [8]:
!gdown https://drive.google.com/uc?id=1o_PM_zB6FxJRdLpEx8lEAhL29IMe1KzI

Downloading...
From: https://drive.google.com/uc?id=1o_PM_zB6FxJRdLpEx8lEAhL29IMe1KzI
To: /content/backend.h5
249MB [00:03, 80.3MB/s]


In [0]:
config_json = """
{
    "model" : {
        "min_input_size":       288,
        "max_input_size":       448,
        "anchors":              [17,21, 21,44, 26,73, 34,28, 40,50, 62,33, 68,84, 99,151, 211,272],
        "labels":               ["Airliner", "Boat", "Bus", "Car", "Chartered Aircraft", "Fighter Aircraft", "Helicopter", "LongVehicle", "Others", "Propeller Aircraft", "Pushback Truck", "Stair Truck", "Trainer Aircraft", "Truck", "Van"]
    },

    "train": {
        "train_image_folder":   "/content/data/images/",
        "train_annot_folder":   "/content/PASCALVOCAnnots/",
        "cache_name":           "/content/drive/My Drive/DeepLearning/ObjectDetection/SavedModels/YOLO_exp/simd_train.pkl",

        "train_times":          8,
        "batch_size":           8,
        "learning_rate":        1e-4,
        "nb_epochs":            5,
        "warmup_epochs":        1,
        "ignore_thresh":        0.5,
        "gpus":                 "0",

        "grid_scales":          [1,1,1],
        "obj_scale":            5,
        "noobj_scale":          1,
        "xywh_scale":           1,
        "class_scale":          1,

        "tensorboard_dir":      "logs",
        "saved_weights_name":   "/content/drive/My Drive/DeepLearning/ObjectDetection/SavedModels/YOLO_exp/simd.h5",
        "debug":                true
    },

    "valid": {
        "valid_image_folder":   "data/yolo_val/images/",
        "valid_annot_folder":   "data/yolo_val/annots/",
        "cache_name":           "/content/drive/My Drive/DeepLearning/ObjectDetection/SavedModels/YOLO_exp/simd_val.pkl",

        "valid_times":          1
    }
}
"""
with open('config.json', 'w') as f:
  f.write(config_json)

In [0]:
!python gen_anchors.py -c config.json

data/images/0001.jpg
data/images/0003.jpg
data/images/0004.jpg
data/images/0005.jpg
data/images/0006.jpg
data/images/0008.jpg
data/images/0010.jpg
data/images/0011.jpg
data/images/0012.jpg
data/images/0014.jpg
data/images/0018.jpg
data/images/0019.jpg
data/images/0020.jpg
data/images/0021.jpg
data/images/0023.jpg
data/images/0024.jpg
data/images/0025.jpg
data/images/0026.jpg
data/images/0028.jpg
data/images/0031.jpg
data/images/0035.jpg
data/images/0036.jpg
data/images/0037.jpg
data/images/0038.jpg
data/images/0039.jpg
data/images/0040.jpg
data/images/0042.jpg
data/images/0043.jpg
data/images/0044.jpg
data/images/0045.jpg
data/images/0046.jpg
data/images/0047.jpg
data/images/0048.jpg
data/images/0050.jpg
data/images/0051.jpg
data/images/0052.jpg
data/images/0053.jpg
data/images/0054.jpg
data/images/0056.jpg
data/images/0058.jpg
data/images/0060.jpg
data/images/0065.jpg
data/images/0066.jpg
data/images/0067.jpg
data/images/0068.jpg
data/images/0069.jpg
data/images/0071.jpg
data/images/0

In [0]:
config = tf.compat.v1.ConfigProto(
    gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.9)
    # device_count = {'GPU': 1}
)
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

def create_training_instances(
    train_annot_folder,
    train_image_folder,
    train_cache,
    valid_annot_folder,
    valid_image_folder,
    valid_cache,
    labels,
):
    # parse annotations of the training set
    train_ints, train_labels = parse_voc_annotation(train_annot_folder, train_image_folder, train_cache, labels)

    # parse annotations of the validation set, if any, otherwise split the training set
    if os.path.exists(valid_annot_folder):
        valid_ints, valid_labels = parse_voc_annotation(valid_annot_folder, valid_image_folder, valid_cache, labels)
    else:
        print("valid_annot_folder not exists. Spliting the trainining set.")

        train_valid_split = int(0.8*len(train_ints))
        np.random.seed(0)
        np.random.shuffle(train_ints)
        np.random.seed()

        valid_ints = train_ints[train_valid_split:]
        train_ints = train_ints[:train_valid_split]

    # compare the seen labels with the given labels in config.json
    if len(labels) > 0:
        overlap_labels = set(labels).intersection(set(train_labels.keys()))

        print('Seen labels: \t'  + str(train_labels) + '\n')
        print('Given labels: \t' + str(labels))

        # return None, None, None if some given label is not in the dataset
        if len(overlap_labels) < len(labels):
            print('Some labels have no annotations! Please revise the list of labels in the config.json.')
            return None, None, None
    else:
        print('No labels are provided. Train on all seen labels.')
        print(train_labels)
        labels = train_labels.keys()

    max_box_per_image = max([len(inst['object']) for inst in (train_ints + valid_ints)])

    return train_ints, valid_ints, sorted(labels), max_box_per_image

def create_callbacks(saved_weights_name, tensorboard_logs, model_to_save):
    makedirs(tensorboard_logs)
    
    early_stop = EarlyStopping(
        monitor     = 'loss', 
        min_delta   = 0.01, 
        patience    = 7, 
        mode        = 'min', 
        verbose     = 1
    )
    checkpoint = CustomModelCheckpoint(
        model_to_save   = model_to_save,
        filepath        = saved_weights_name,# + '{epoch:02d}.h5', 
        monitor         = 'loss', 
        verbose         = 1, 
        save_best_only  = True, 
        mode            = 'min', 
        period          = 1
    )
    reduce_on_plateau = ReduceLROnPlateau(
        monitor  = 'loss',
        factor   = 0.1,
        patience = 2,
        verbose  = 1,
        mode     = 'min',
        epsilon  = 0.01,
        cooldown = 0,
        min_lr   = 0
    )
    tensorboard = CustomTensorBoard(
        log_dir                = tensorboard_logs,
        write_graph            = True,
        write_images           = True,
    )    

    csv_logger = CSVLogger("/content/drive/My Drive/DeepLearning/ObjectDetection/SavedHistory/yolo_exp/hist.csv", append=True)

    return [early_stop, checkpoint, reduce_on_plateau, tensorboard, csv_logger]

def create_model(
    nb_class, 
    anchors, 
    max_box_per_image, 
    max_grid, batch_size, 
    warmup_batches, 
    ignore_thresh, 
    multi_gpu, 
    saved_weights_name, 
    lr,
    grid_scales,
    obj_scale,
    noobj_scale,
    xywh_scale,
    class_scale  
):
    if multi_gpu > 1:
        with tf.device('/cpu:0'):
            template_model, infer_model = create_yolov3_model(
                nb_class            = nb_class, 
                anchors             = anchors, 
                max_box_per_image   = max_box_per_image, 
                max_grid            = max_grid, 
                batch_size          = batch_size//multi_gpu, 
                warmup_batches      = warmup_batches,
                ignore_thresh       = ignore_thresh,
                grid_scales         = grid_scales,
                obj_scale           = obj_scale,
                noobj_scale         = noobj_scale,
                xywh_scale          = xywh_scale,
                class_scale         = class_scale
            )
    else:
        template_model, infer_model = create_yolov3_model(
            nb_class            = nb_class, 
            anchors             = anchors, 
            max_box_per_image   = max_box_per_image, 
            max_grid            = max_grid, 
            batch_size          = batch_size, 
            warmup_batches      = warmup_batches,
            ignore_thresh       = ignore_thresh,
            grid_scales         = grid_scales,
            obj_scale           = obj_scale,
            noobj_scale         = noobj_scale,
            xywh_scale          = xywh_scale,
            class_scale         = class_scale
        )  

    # load the pretrained weight if exists, otherwise load the backend weight only
    if os.path.exists(saved_weights_name): 
        print("\nLoading pretrained weights.\n")
        template_model.load_weights(saved_weights_name)
    else:
        template_model.load_weights("backend.h5", by_name=True)       

    if multi_gpu > 1:
        train_model = multi_gpu_model(template_model, gpus=multi_gpu)
    else:
        train_model = template_model      

    optimizer = Adam(lr=lr, clipnorm=0.001)
    train_model.compile(loss=dummy_loss, optimizer=optimizer)             

    return train_model, infer_model

def _main_(path):
    config_path = path

    with open(config_path) as config_buffer:    
        config = json.loads(config_buffer.read())

    ###############################
    #   Parse the annotations 
    ###############################
    train_ints, valid_ints, labels, max_box_per_image = create_training_instances(
        config['train']['train_annot_folder'],
        config['train']['train_image_folder'],
        config['train']['cache_name'],
        config['valid']['valid_annot_folder'],
        config['valid']['valid_image_folder'],
        config['valid']['cache_name'],
        config['model']['labels']
    )
    print('\nTraining on: \t' + str(labels) + '\n')

    ###############################
    #   Create the generators 
    ###############################    
    train_generator = BatchGenerator(
        instances           = train_ints, 
        anchors             = config['model']['anchors'],   
        labels              = labels,        
        downsample          = 32, # ratio between network input's size and network output's size, 32 for YOLOv3
        max_box_per_image   = max_box_per_image,
        batch_size          = config['train']['batch_size'],
        min_net_size        = config['model']['min_input_size'],
        max_net_size        = config['model']['max_input_size'],   
        shuffle             = True, 
        jitter              = 0.3, 
        norm                = normalize
    )
    
    valid_generator = BatchGenerator(
        instances           = valid_ints, 
        anchors             = config['model']['anchors'],   
        labels              = labels,        
        downsample          = 32, # ratio between network input's size and network output's size, 32 for YOLOv3
        max_box_per_image   = max_box_per_image,
        batch_size          = config['train']['batch_size'],
        min_net_size        = config['model']['min_input_size'],
        max_net_size        = config['model']['max_input_size'],   
        shuffle             = True, 
        jitter              = 0.0, 
        norm                = normalize
    )

    ###############################
    #   Create the model 
    ###############################
    if os.path.exists(config['train']['saved_weights_name']): 
        config['train']['warmup_epochs'] = 0
    warmup_batches = config['train']['warmup_epochs'] * (config['train']['train_times']*len(train_generator))   

    os.environ['CUDA_VISIBLE_DEVICES'] = config['train']['gpus']
    multi_gpu = len(config['train']['gpus'].split(','))

    train_model, infer_model = create_model(
        nb_class            = len(labels), 
        anchors             = config['model']['anchors'], 
        max_box_per_image   = max_box_per_image, 
        max_grid            = [config['model']['max_input_size'], config['model']['max_input_size']], 
        batch_size          = config['train']['batch_size'], 
        warmup_batches      = warmup_batches,
        ignore_thresh       = config['train']['ignore_thresh'],
        multi_gpu           = multi_gpu,
        saved_weights_name  = config['train']['saved_weights_name'],
        lr                  = config['train']['learning_rate'],
        grid_scales         = config['train']['grid_scales'],
        obj_scale           = config['train']['obj_scale'],
        noobj_scale         = config['train']['noobj_scale'],
        xywh_scale          = config['train']['xywh_scale'],
        class_scale         = config['train']['class_scale'],
    )

    ###############################
    #   Kick off the training
    ###############################
    callbacks = create_callbacks(config['train']['saved_weights_name'], config['train']['tensorboard_dir'], infer_model)

    history = train_model.fit_generator(
        generator        = train_generator, 
        steps_per_epoch  = len(train_generator) * config['train']['train_times'], 
        epochs           = config['train']['nb_epochs'] + config['train']['warmup_epochs'], 
        # verbose          = 2 if config['train']['debug'] else 1,
        verbose          = 2,
        callbacks        = callbacks, 
        workers          = 4,
        max_queue_size   = 8
    )

    # make a GPU version of infer_model for evaluation
    if multi_gpu > 1:
        infer_model = load_model(config['train']['saved_weights_name'])

    ###############################
    #   Run the evaluation
    ###############################   
    # compute mAP for all the classes
    average_precisions = evaluate(infer_model, valid_generator)

    # print the score
    for label, average_precision in average_precisions.items():
        print(labels[label] + ': {:.4f}'.format(average_precision))
    print('mAP: {:.4f}'.format(sum(average_precisions.values()) / len(average_precisions)))

    return history

In [12]:
history = _main_('config.json')

Seen labels: 	{'Car': 13469, 'Van': 3838, 'Truck': 1906, 'Others': 512, 'Airliner': 664, 'Stair Truck': 297, 'Pushback Truck': 118, 'Propeller Aircraft': 130, 'Chartered Aircraft': 411, 'Helicopter': 42, 'Bus': 1250, 'LongVehicle': 1081, 'Boat': 5647, 'Trainer Aircraft': 432, 'Fighter Aircraft': 37}

Given labels: 	['Airliner', 'Boat', 'Bus', 'Car', 'Chartered Aircraft', 'Fighter Aircraft', 'Helicopter', 'LongVehicle', 'Others', 'Propeller Aircraft', 'Pushback Truck', 'Stair Truck', 'Trainer Aircraft', 'Truck', 'Van']

Training on: 	['Airliner', 'Boat', 'Bus', 'Car', 'Chartered Aircraft', 'Fighter Aircraft', 'Helicopter', 'LongVehicle', 'Others', 'Propeller Aircraft', 'Pushback Truck', 'Stair Truck', 'Trainer Aircraft', 'Truck', 'Van']



/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:998: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '







resizing:  320 320
Epoch 1/6

resizing:  448 448
resizing:  384 384
resizing:  448 448
resizing:  384 384
resizing:  352 352
resizing:  448 448
resizing:  320 320
resizing:  416 416
resizing:  320 320
resizing:  448 448
resizing:  416 416
resizing:  448 448
resizing:  416 416
resizing:  352 352
resizing:  448 448
resizing:  288 288
resizing:  384 384
resizing:  352 352
resizing:  448 448
resizing:  448 448
resizing:  352 352
resizing:  320 320
resizing:  320 320
resizing:  288 288
resizing:  416 416
resizing:  320 320
resizing:  384 384
resizing:  288 288
resizing:  384 384
resizing:  448 448
resizing:  416 416
resizing:  320 320
resizing:  288 288
resizing:  448 448
resizing:  288 288
resizing:  352 352
resizing:  448 448
resizing:  320 320
resizing:  448 448
resizing:  448 448
resizing:  288 288
resizing:  448 448
resizing:  384 384
resizing:  352 352
resizing:  384 384
resizing:  448 448
resizing:  352 352
resizing:  320 320
resizing:  320 320
resizing:  448 448
resizing:  320 

In [0]:
# !python train.py -c config.json

Using TensorFlow backend.
2020-05-14 06:47:16.843406: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2020-05-14 06:47:16.843681: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1dc0a00 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-05-14 06:47:16.843717: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-05-14 06:47:16.845884: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-05-14 06:47:16.924658: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-05-14 06:47:16.925642: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1dc0bc0 initialized for platform CUDA (this does not guarantee that XLA will be used). De

In [0]:
!python predict.py -c config.json -i /content/data/images/0007.jpg

In [0]:
d = {"bus": 1286, 'car': 13747, 'van': 3735, 'truck': 1755, 'other': 426, 'airliner': 666, 'stairtruck': 299, 'pushbacktruck': 122, 'longvehicle': 1034, 'propeller': 123, 'chartered': 412, 'helicopter': 36, 'trainer': 403, 'boat': 6051, 'fighter': 35, 'pushback': 15, 'others': 119}
print(sorted(list(d.keys())))

['airliner', 'boat', 'bus', 'car', 'chartered', 'fighter', 'helicopter', 'longvehicle', 'other', 'others', 'propeller', 'pushback', 'pushbacktruck', 'stairtruck', 'trainer', 'truck', 'van']


In [0]:
import cv2

In [0]:
i = cv2.imread('data/images/1314.jpg')

array([[[ 34,  31,  33],
        [ 21,  18,  20],
        [ 20,  17,  19],
        ...,
        [147, 143, 142],
        [ 83,  77,  82],
        [ 53,  46,  53]],

       [[ 29,  26,  28],
        [ 13,  10,  12],
        [  8,   5,   7],
        ...,
        [ 78,  74,  73],
        [ 33,  27,  32],
        [ 24,  17,  24]],

       [[ 20,  17,  19],
        [ 11,   8,  10],
        [  5,   2,   4],
        ...,
        [ 28,  23,  20],
        [  5,   0,   1],
        [ 10,   3,   8]],

       ...,

       [[ 54,  63,  53],
        [ 56,  65,  55],
        [ 58,  66,  59],
        ...,
        [105, 104, 108],
        [112, 111, 113],
        [ 91,  90,  92]],

       [[ 54,  63,  53],
        [ 55,  64,  54],
        [ 56,  64,  57],
        ...,
        [ 92,  91,  95],
        [135, 134, 138],
        [150, 149, 153]],

       [[ 56,  65,  55],
        [ 56,  65,  55],
        [ 56,  64,  57],
        ...,
        [ 82,  81,  85],
        [139, 138, 142],
        [159, 158, 162]]

In [0]:
!cp kangaroo.h5 '/content/drive/My Drive/DeepLearning/ObjectDetection/SavedModels/YOLO_exp/'

cp: target 'Drive/DeepLearning/ObjectDetection/SavedModels/YOLO_exp/' is not a directory


In [0]:
!cp kangaroo.h5 '/content/drive/My Drive/DeepLearning/ObjectDetection/SavedModels/YOLO_exp/'
!cp kangaroo_train.pkl '/content/drive/My Drive/DeepLearning/ObjectDetection/SavedModels/YOLO_exp/'
!cp validate.pkl '/content/drive/My Drive/DeepLearning/ObjectDetection/SavedModels/YOLO_exp/'

In [13]:
!python evaluate.py -c config.json

Using TensorFlow backend.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
2020-05-19 15:32:11.455937: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-05-19 15:32:11.458810: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-05-19 15:32:11.459148: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
2020-05-19 15:32:11.459470: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-05-19 15:32:11.462390: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2020-05-19 15:32:11.475078: I tensorfl